In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
import sys

SOURCE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__name__)))
sys.path.insert(0, SOURCE_DIR)

In [3]:
import tensorflow as tf

tf.compat.v1.enable_eager_execution()

In [4]:
import malaya_speech
import malaya_speech.config
from malaya_speech.train.model import glowtts
import tensorflow as tf
import numpy as np






The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [ ]:
config = malaya_speech.config.fastspeech2_config
config = glowtts.Config(vocab_size = 66, **config)
config_glowtts = glowtts.Config_GlowTTS(malaya_speech.config.glowtts_config)

In [ ]:
i = tf.placeholder(tf.int32, [None, None])
lens = tf.placeholder(tf.int32, [None, None])

In [ ]:
encoder = glowtts.Encoder(config, config_glowtts)

In [ ]:
x_m, x_logs, logw, x_mask = encoder(i)

In [ ]:
x_m, x_logs, logw, x_mask

In [ ]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [ ]:
r = sess.run([x_m, x_logs, logw, x_mask], 
         feed_dict = {i: [[1,2,3]]})

In [ ]:
mel_before, mel_after, duration_outputs, pitch_outputs = model(i, lens, pitchs, training = True)

In [ ]:
mel_before, mel_after, duration_outputs, pitch_outputs

In [ ]:
loss_f = tf.losses.mean_squared_error

In [ ]:
log_duration = tf.math.log(
    tf.cast(tf.math.add(lens, 1), tf.float32)
)
duration_loss = loss_f(log_duration, duration_outputs)

In [ ]:
max_length = tf.cast(tf.reduce_max(mel_lengths), tf.int32)
mask = tf.sequence_mask(
    lengths = mel_lengths, maxlen = max_length, dtype = tf.float32
)
mask = tf.expand_dims(mask, axis = -1)
mask

In [ ]:
from functools import partial
from malaya_speech.train.loss import calculate_2d_loss, calculate_3d_loss

mse_mel = partial(
    loss_f,
    weights = mask
)

In [ ]:
mel_loss_before = calculate_3d_loss(mel_outputs, mel_before, mse_mel)
mel_loss_before

In [ ]:
mel_loss_after = calculate_3d_loss(mel_outputs, mel_after, mse_mel)
mel_loss_after

In [ ]:
max_length = tf.cast(tf.reduce_max(pitchs_lengths), tf.int32)
mask = tf.sequence_mask(
    lengths = pitchs_lengths, maxlen = max_length, dtype = tf.float32
)
pitchs_mel = partial(
    loss_f,
    weights = mask
)
pitchs_loss = calculate_2d_loss(pitchs, pitch_outputs, pitchs_mel)

In [ ]:
loss = duration_loss + mel_loss_before + mel_loss_after + pitchs_loss

In [ ]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [ ]:
import pickle

with open('dataset-mel.pkl', 'rb') as fopen:
    data, d = pickle.load(fopen)
    
with open('dataset-mel-wav.pkl', 'rb') as fopen:
    wav = pickle.load(fopen)

In [ ]:
data.keys()

In [ ]:
d

In [ ]:
data['text_ids'].shape

In [ ]:
def average_by_duration(x, durs):
    mel_len = durs.sum()
    durs_cum = np.cumsum(np.pad(durs, (1, 0)))
    
    x_char = np.zeros((durs.shape[0],), dtype=np.float32)
    for idx, start, end in zip(range(mel_len), durs_cum[:-1], durs_cum[1:]):
        values = x[start:end][np.where(x[start:end] != 0.0)[0]]
        x_char[idx] = np.mean(values) if len(values) > 0 else 0.0

    return x_char.astype(np.float32)

In [ ]:
f0 = average_by_duration(data['f0'][0], d)
energy = average_by_duration(data['energy'][0], d)

In [ ]:
mel_len = data['mel'].shape[1]
snd = parselmouth.Sound(wav['wav'][0],sampling_frequency=22050)
pitch = snd.to_pitch(time_step=snd.duration / (mel_len + 3)
                         ).selected_array['frequency']
pitch.shape, mel_len

In [ ]:
pitch = average_by_duration(pitch, d)
pitch.shape

In [ ]:
r = sess.run([mel_before, mel_after, duration_outputs, pitch_outputs], 
         feed_dict = {i: data['text_ids'],
                      lens: [d],
                      pitchs: [pitch]})

In [ ]:
r[0].shape, r[1].shape, r[2].shape, r[3].shape

In [ ]:
r = sess.run([duration_loss, mel_loss_before, mel_loss_after, pitchs_loss], 
         feed_dict = {i: data['text_ids'],
                      lens: [d],
                      mel_outputs:data['mel'],
                      mel_lengths:data['len_mel'][0],
                      pitchs: [pitch],
                      pitchs_lengths: [len(pitch)],})

In [ ]:
r